# Emergent vs Baseline Ablation Visualization

This notebook visualizes the results of the corridor-width ablation experiment comparing emergent signals vs baseline geometric ranking.

In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import numpy as np

# Set style
plt.style.use('default')
sns.set_palette("husl")

# Load summary
with open('ablation_summary.json', 'r') as f:
    summary = json.load(f)

gates = list(summary['gate_results'].keys())
baseline_ranks = [summary['gate_results'][g]['baseline_rank'] for g in gates]
emergent_ranks = [summary['gate_results'][g]['emergent_rank'] for g in gates]

## 1. Rank Comparison: Baseline vs Emergent

In [ ]:
# Bar chart of ranks
x = np.arange(len(gates))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, baseline_ranks, width, label='Baseline', alpha=0.8)
bars2 = ax.bar(x + width/2, emergent_ranks, width, label='Emergent', alpha=0.8)

ax.set_ylabel('Rank of True Factor (lower is better)')
ax.set_title('Rank Comparison: Baseline vs Emergent Dynamics')
ax.set_xticks(x)
ax.set_xticklabels(gates)
ax.legend()
ax.set_yscale('log')

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom')

for bar in bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('figures/rank_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. Corridor Width Comparison

In [ ]:
# Corridor width: number of candidates with better rank than p
baseline_widths = [r - 1 for r in baseline_ranks]  # candidates above p
emergent_widths = [r - 1 for r in emergent_ranks]

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, baseline_widths, width, label='Baseline', alpha=0.8)
bars2 = ax.bar(x + width/2, emergent_widths, width, label='Emergent', alpha=0.8)

ax.set_ylabel('Corridor Width (# candidates above true factor)')
ax.set_title('Corridor Width Comparison')
ax.set_xticks(x)
ax.set_xticklabels(gates)
ax.legend()
ax.set_yscale('log')

for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom')

for bar in bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('figures/corridor_width.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Time Series Analysis

Load detailed logs for one gate and show time series.

In [ ]:
# Load detailed log for G100
with open('logs/ablation_G100.json', 'r') as f:
    g100_data = json.load(f)

time_series = g100_data['emergent']['time_series']
sortedness = time_series['sortedness']
aggregation = time_series['aggregation']
dg_index = time_series['dg_index']

steps = list(range(len(sortedness)))

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# Sortedness
ax1.plot(steps, sortedness, 'b-', linewidth=2, label='Sortedness')
ax1.set_ylabel('Sortedness')
ax1.set_title('Time Series: Emergent Signals (G100)')
ax1.grid(True, alpha=0.3)

# Aggregation
ax2.plot(steps, aggregation, 'g-', linewidth=2, label='Aggregation')
ax2.set_ylabel('Aggregation')
ax2.grid(True, alpha=0.3)

# DG Index
ax3.plot(steps, dg_index, 'r-', linewidth=2, label='DG Index')
ax3.set_ylabel('DG Index')
ax3.set_xlabel('Swap Step')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('figures/time_series_G100.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Algotype Distribution Heatmap

Show spatial distribution of algotypes near the true factor's final position.

In [ ]:
# Get final state
final_state = g100_data['emergent']['final_state']
p_true = g100_data['p_true']

# Find p's position
p_index = next(i for i, cell in enumerate(final_state) if int(cell['n']) == p_true)

# Get nearby cells (say ±20)
start_idx = max(0, p_index - 20)
end_idx = min(len(final_state), p_index + 21)
nearby = final_state[start_idx:end_idx]

# Count algotypes
algotypes = [cell['algotype'] for cell in nearby]
positions = list(range(start_idx, end_idx))

# Create heatmap data
unique_algos = list(set(algotypes))
algo_to_idx = {algo: i for i, algo in enumerate(unique_algos)}

heatmap_data = np.zeros((len(unique_algos), len(positions)))
for pos_idx, algo in enumerate(algotypes):
    heatmap_data[algo_to_idx[algo], pos_idx] = 1

# Plot
fig, ax = plt.subplots(figsize=(12, 4))
im = ax.imshow(heatmap_data, aspect='auto', cmap='Blues')

ax.set_yticks(range(len(unique_algos)))
ax.set_yticklabels(unique_algos)
ax.set_xlabel('Position Index')
ax.set_title(f'Algotype Distribution near True Factor (G100, position {p_index})')

# Mark p's position
p_pos_in_nearby = p_index - start_idx
ax.axvline(p_pos_in_nearby, color='red', linestyle='--', linewidth=2, label='True Factor')
ax.legend()

plt.tight_layout()
plt.savefig('figures/algotype_heatmap_G100.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary Statistics

In [ ]:
# Compute statistics
rank_improvements = [b - e for b, e in zip(baseline_ranks, emergent_ranks)]
avg_improvement = np.mean(rank_improvements)
std_improvement = np.std(rank_improvements)

print(f"Average Rank Improvement: {avg_improvement:.1f} ± {std_improvement:.1f}")
print(f"Improvement > 0: {sum(1 for r in rank_improvements if r > 0)} / {len(rank_improvements)} gates")

# Success criterion: >20% rank reduction
total_candidates = summary['experiment_config']['candidate_count']  # approximate
percent_reductions = [(b - e) / b * 100 for b, e in zip(baseline_ranks, emergent_ranks)]
avg_percent = np.mean(percent_reductions)
print(f"Average Percent Rank Reduction: {avg_percent:.1f}%")
print(f"Success (>20% reduction): {sum(1 for p in percent_reductions if p > 20)} / {len(percent_reductions)} gates")